In [1]:
import numpy as np
import pandas as pd

In [2]:
#Zaciagniecie baz danych
obciazenie_df = pd.read_csv("C://Users//ksobo//Desktop//DS_final_project//Zlecenia.csv", sep=';')
godziny_df = pd.read_csv("C://Users//ksobo//Desktop//DS_final_project//Godziny.csv", sep=';')

In [3]:
#Ekstrakcja danych i organizacja danych - pozostawienie tylko kolumn wymaganych, pozostawienie tylko projektów "w toku" - nie działa
obciazenie_df1 = obciazenie_df.drop(columns=['work_preparation_Plan','work_preparation_done'], axis=1)
obciazenie_df2 = obciazenie_df1.loc[obciazenie_df1['Active'].isin(['Active'])]

In [4]:
#Left join po No z godzinami
obciazenie_df2 = pd.merge(obciazenie_df2, godziny_df, how='left', on='No')

#TEST!!!!! wypelnienie NaN zerami
#obciazenie_df2['Wszystkie.technol'] = obciazenie_df2['Wszystkie.technol'].fillna(0)
#obciazenie_df2['Wszystkie.wykonane'] = obciazenie_df2['Wszystkie.wykonane'].fillna(0)

In [5]:
#Zdefiniowanie tygodnia obecnego
#przyjmujemy obecny jako 202112(tydzień zaciągnięcia danych),
#docelowo może zostać zamieniony funkcją obliczającą jak poniżej
#WPISAĆ FUNKCJĘ!!!!!!!!!

current_week = 202112



In [6]:
#Zmiana terminu rozpoczęcia oraz zakończenia na obecny jeżeli miało mieć ono miejsce w przeszłości
# !!!NIE DZIAŁA!!!!

#if  obciazenie_df2['Finnish_week'] < current_week:
#    obciazenie_df2['Finnish_week'] == current_week
#else:
#    obciazenie_df2['Finnish_week'] == obciazenie_df2['Finnish_week']
    
#if  obciazenie_df2['Pre_fabrication_plan'] < current_week:
#    obciazenie_df2['Pre_fabrication_plan'] = current_week
#else:
#    obciazenie_df2['Pre_fabrication_plan'] = obciazenie_df2['Finnish_week']

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
#Zastapienie NaN w godzinach wartościami przewidzianymi z prekalkulacji

#A - dziób(wędka)
#Labor rate	130 rbg/1000kg
#Assembling	0,4
#Welding	0,33
#Grindining	0,07
#Else		0,2

#B - belka(środek)
#Labor rate	100 rbg/1000kg
#Assembling	0,3
#Welding	0,4
#Grindining	0,2
#Else		0,1

#C - końcóka(środek)
#Labor rate	120 rbg/1000kg
#Assembling	0,25
#Welding	0,4
#Grindining	0,2
#Else		0,15

# IF Wszystkie technol = NaN -> Weight for 1 * Quantity * Typ (IF A - 0,13 elif B - 0,1 elif C - 0,12)

In [ ]:
#Obliczenie kolumn - pozostałego czasu na produkcję oraz pozostałych godzin jeżeli >52 to odjąc od wyniku ilość tyg w danym roku - def 52
obciazenie_df2['Weeks_left'] = (obciazenie_df2['Finnish_week']) - (obciazenie_df2['Pre_fabrication_plan'])

In [ ]:
#???????Czy liczyć nadwyżkę 20% jako zabezpieczenie przy przekroczeniach??????
obciazenie_df2['Work_h_left'] = (obciazenie_df2['Wszystkie.technol']) - (obciazenie_df2['Wszystkie.wykonane'])

#Jeżeli zlecenie zlecenie przekroczyło godziny praca liczona jest w nadgodzinach - przyjmuje się 0 do obciażenia

In [ ]:
#Podzielenie pozostałej pracy równomiernie na tygodnie
obciazenie_df2['Workload_per_week'] = (obciazenie_df2['Work_h_left']) / (obciazenie_df2['Weeks_left'])
obciazenie_df2['Workload_per_week']

In [ ]:
#Podział obciążenia na tygodnie obecny+13 tyg
obciazenie_df3 = obciazenie_df2[["No", "Workload_per_week"]]

x = current_week
for i in range(0,13):
    obciazenie_df3[x+i]=0

obciazenie_df3

In [ ]:
#Dodanie obciazenia zarezerwowanego na dany tydzien w zależności od rozpoczecia i zakonczenia danego zlecenia
#If prefabrication plan is >column index & <= finnish week -> column index.No zlecenia = Workload_per_week

In [ ]:
#Zrobienie wykresu